# learning with the docs

Docs文档中对每一个模块的讲解

In [20]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

## 一、Autograd mechanics—— autograd中的机制

Variable中有两个 flags `requires_grad` 和 `volatile`

+ requires_grad
operation中有一个Variablle有grad为T，则output的Variable就可以被计算grad；
如果二者都不需要计算grad，其生成的output variable也不会在backward过程中被计算grad

In [21]:
x= Variable(torch.randn(5, 5))
y = Variable(torch.randn(5,5))
z = Variable(torch.randn(5,5), requires_grad=True)

x.requires_grad

False

In [22]:
z.requires_grad

True

In [23]:
a = x + y
a.requires_grad

False

In [24]:
b = z + x
b.requires_grad  # b需要被计算梯度

True

In [25]:
import torchvision
import torch.optim as optim
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
# Replace the last fully-connected layer
# Parameters of newly constructed modules have requires_grad=True by default
model.fc = nn.Linear(512, 100)

# Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

+ volatile
当你确定不需要用backward的时候，它会使用尽可能小的内存，同时所有的require_grad都会被设置为False.
volatile与requires_grad的区别在于二者的传播方式；只要有一个volatile为true，后面的子图都不必计算梯度了。

In [35]:
 regular_input = Variable(torch.randn(5, 5))
volatile_input = Variable(torch.randn(5, 5), volatile=True)
model = torchvision.models.resnet18(pretrained=True)

In [27]:
volatile_input

Variable containing:
-0.4512  0.0196  1.0143 -1.3464 -0.6074
-0.3315 -0.0032 -1.5006  1.1204 -1.9988
 2.5679 -0.8157 -1.2020  1.5336 -0.7697
-0.6934 -1.3897 -0.0491 -0.2404 -0.7632
-0.7187 -0.5765 -0.5374  0.6227 -0.7102
[torch.FloatTensor of size 5x5]

### autograd 是怎么记录历史的呢?
   每个Variable有一个.creator的属性，指向上一个产生它的函数；
每当新生成一个新的Variable时，一个function就被实例化了，forward函数被执行，它的输出Variable的creator就指向这个function。

计算图在每一次迭代过程中都会被重新创建

### In-place operations on Variable
Unless you’re operating under heavy memory pressure, you might never need to use them.

## Serialization semantics 序列化的语义

### 1. 保存一个model

+ (推荐的) 保存以及只载入模型参数


+ 保存整个模型

## 文档

### 1. Tensor


In [39]:
x = torch.Tensor(torch.randn(3, 2))
x


 0.6338  2.5317
-1.2868 -0.1617
-1.5095 -0.1871
[torch.FloatTensor of size 3x2]

In [42]:
torch.is_tensor(x)  # 是不是tensor

True

In [43]:
torch.is_storage(x) # 是不是一个pytorch storage object

False

In [44]:
# 计算一共有多少个元素
torch.numel(x)

6

**Creation Ops**

In [46]:
torch.eye(5, 2)  # 5 行 2 列


 1  0
 0  1
 0  0
 0  0
 0  0
[torch.FloatTensor of size 5x2]

In [49]:
torch.eye(5, 2, out= x)
x


 1  0
 0  1
 0  0
 0  0
 0  0
[torch.FloatTensor of size 5x2]

In [55]:
import numpy as np
a=np.array([1,2,3])
x=torch.from_numpy(a)
x


 1
 2
 3
[torch.LongTensor of size 3]

In [56]:
x[0] = 100
a

array([100,   2,   3])

In [58]:
torch.linspace(start = 1, end = 100, steps = 10)  # steps表示在这个区间内等距抽样的个数


   1
  12
  23
  34
  45
  56
  67
  78
  89
 100
[torch.FloatTensor of size 10]

$10^{start}到10^{end}之间$

In [59]:
torch.logspace( start = 1, end = 4, steps = 10)


    10.0000
    21.5443
    46.4159
   100.0000
   215.4435
   464.1587
  1000.0000
  2154.4343
  4641.5898
 10000.0000
[torch.FloatTensor of size 10]

In [60]:
torch.ones(2,5)


 1  1  1  1  1
 1  1  1  1  1
[torch.FloatTensor of size 2x5]

In [61]:
# 随机数 【0,1)均匀分布
torch.rand(2,3)


 0.5443  0.2540  0.9704
 0.2439  0.8733  0.8033
[torch.FloatTensor of size 2x3]

In [62]:
# 标准正态分布
torch.randn(4)


-1.5329
 0.4362
-1.0002
-0.7377
[torch.FloatTensor of size 4]

In [64]:
# 随机排列 0到n-1
torch.randperm(5)


 0
 3
 2
 4
 1
[torch.LongTensor of size 5]

In [68]:
torch.range(1, 4, step = 1)


 1
 2
 3
 4
[torch.FloatTensor of size 4]

In [69]:
torch.zeros(2,3)


 0  0  0
 0  0  0
[torch.FloatTensor of size 2x3]

**索引，切片，join， mutating**
+ 拼接

In [72]:
x = torch.randn(2,3)
x


-0.7082 -2.0255 -1.4898
-0.0942  0.8270 -0.0154
[torch.FloatTensor of size 2x3]

In [74]:
torch.cat((x,x,x), 0) # rbind


-0.7082 -2.0255 -1.4898
-0.0942  0.8270 -0.0154
-0.7082 -2.0255 -1.4898
-0.0942  0.8270 -0.0154
-0.7082 -2.0255 -1.4898
-0.0942  0.8270 -0.0154
[torch.FloatTensor of size 6x3]

In [75]:
torch.cat((x,x,x), 1)


-0.7082 -2.0255 -1.4898 -0.7082 -2.0255 -1.4898 -0.7082 -2.0255 -1.4898
-0.0942  0.8270 -0.0154 -0.0942  0.8270 -0.0154 -0.0942  0.8270 -0.0154
[torch.FloatTensor of size 2x9]

In [78]:
y = torch.cat((x,x,x), 1)
torch.chunk(y, 2)

(
 -0.7082 -2.0255 -1.4898 -0.7082 -2.0255 -1.4898 -0.7082 -2.0255 -1.4898
 [torch.FloatTensor of size 1x9], 
 -0.0942  0.8270 -0.0154 -0.0942  0.8270 -0.0154 -0.0942  0.8270 -0.0154
 [torch.FloatTensor of size 1x9])

In [79]:
torch.chunk(y, 3, dim=1)

(
 -0.7082 -2.0255 -1.4898
 -0.0942  0.8270 -0.0154
 [torch.FloatTensor of size 2x3], 
 -0.7082 -2.0255 -1.4898
 -0.0942  0.8270 -0.0154
 [torch.FloatTensor of size 2x3], 
 -0.7082 -2.0255 -1.4898
 -0.0942  0.8270 -0.0154
 [torch.FloatTensor of size 2x3])